<a href="https://colab.research.google.com/github/ecutolo/notebooks/blob/master/DLGD_PL_resnet_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision sklearn 

# Adjusting ResNet for MNIST in PyTorch

For detailed description, go to: https://zablo.net/blog/post/using-resnet-for-mnist-in-pytorch-tutorial

In [ ]:
from torchvision.models.resnet import ResNet, BasicBlock
from torchvision.datasets import MNIST
from tqdm.autonotebook import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import inspect
import time
from torch import nn, optim
import torch
from torchvision.transforms import Compose, ToTensor, Normalize, Resize
from torch.utils.data import DataLoader

In [ ]:
class MnistResNet(ResNet):
    def __init__(self):
        super(MnistResNet, self).__init__(BasicBlock, [2, 2, 2, 2], num_classes=10)
        self.conv1 = torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        
    def forward(self, x):
        return torch.softmax(super(MnistResNet, self).forward(x), dim=-1)


In [ ]:
def get_data_loaders(train_batch_size, val_batch_size):
    mnist = MNIST(download=True, train=True, root=".").train_data.float()
    
    data_transform = Compose([ Resize((224, 224)),ToTensor(), Normalize((mnist.mean()/255,), (mnist.std()/255,))])
    train_ds = MNIST(download=True, root=".", transform=data_transform, train=True)

    val_ds = MNIST(download=True, root=".", transform=data_transform, train=False)

    train_loader = DataLoader(train_ds, batch_size=train_batch_size, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=val_batch_size, shuffle=False)
    return train_loader, val_loader

In [ ]:
def calculate_metric(metric_fn, true_y, pred_y):
    if "average" in inspect.getfullargspec(metric_fn).args:
        return metric_fn(true_y, pred_y, average="macro")
    else:
        return metric_fn(true_y, pred_y)
    
def print_scores(p, r, f1, a, batch_size):
    for name, scores in zip(("precision", "recall", "F1", "accuracy"), (p, r, f1, a)):
        print(f"\t{name.rjust(14, ' ')}: {sum(scores)/batch_size:.4f}")

In [ ]:
start_ts = time.time()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
epochs = 1

model = MnistResNet().to(device)
train_loader, val_loader = get_data_loaders(256, 256)

losses = []
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adadelta(model.parameters())

batches = len(train_loader)
val_batches = len(val_loader)

# training loop + eval loop
for epoch in range(epochs):
    total_loss = 0
    progress = tqdm(enumerate(train_loader), desc="Loss: ", total=batches)
    model.train()
    
    for i, data in progress:
        X, y = data[0].to(device), data[1].to(device)
        
        model.zero_grad()
        outputs = model(X)
        loss = loss_function(outputs, y)

        loss.backward()
        optimizer.step()
        current_loss = loss.item()
        total_loss += current_loss
        progress.set_description("Loss: {:.4f}".format(total_loss/(i+1)))
        
    torch.cuda.empty_cache()
    
    val_losses = 0
    precision, recall, f1, accuracy = [], [], [], []
    
    model.eval()
    with torch.no_grad():
        for i, data in enumerate(val_loader):
            X, y = data[0].to(device), data[1].to(device)
            outputs = model(X)
            val_losses += loss_function(outputs, y)

            predicted_classes = torch.max(outputs, 1)[1]
            
            for acc, metric in zip((precision, recall, f1, accuracy), 
                                   (precision_score, recall_score, f1_score, accuracy_score)):
                acc.append(
                    calculate_metric(metric, y.cpu(), predicted_classes.cpu())
                )
        
        
        
    print(f"Epoch {epoch+1}/{epochs}, training loss: {total_loss/batches}, validation loss: {val_losses/val_batches}")
    print_scores(precision, recall, f1, accuracy, val_batches)
    losses.append(total_loss/batches)
print(losses)
print(f"Training time: {time.time()-start_ts}s")

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

...

  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:62: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")


Loss:   0%|          | 0/235 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
!pip install pytorch-lightning


     |████████████████████████████████| 1.0 MB 4.2 MB/s 
     |████████████████████████████████| 596 kB 42.9 MB/s 
     |████████████████████████████████| 829 kB 34.9 MB/s 
     |████████████████████████████████| 329 kB 44.3 MB/s 
     |████████████████████████████████| 132 kB 46.6 MB/s 
     |████████████████████████████████| 1.1 MB 21.7 MB/s 
     |████████████████████████████████| 192 kB 41.8 MB/s 
     |████████████████████████████████| 271 kB 46.6 MB/s 
     |████████████████████████████████| 160 kB 38.5 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=91703792ee73ddfd562086d7bf5ddfd2881950164838e0d0198b8da7e6d0c140
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: future
    Found exis

### TODO 1: implement lightning module

In [ ]:
import pytorch_lightning as pl


class MnistLitModel(pl.LightningModule):
  # Code goes here
  ...

trainer = pl.Trainer(max_epochs=1, gpus=1)

# training
model = MnistLitModel()
trainer.fit(model=model, train_dataloaders=train_loader)

# testing
trainer.test(model=model, test_dataloaders=val_loader)


In [ ]:
import pytorch_lightning as pl


class MnistLitModel(pl.LightningModule):
  def __init__(self):
    super().__init__()
    self.mod = MnistResNet()
    self.loss_function = nn.CrossEntropyLoss()

  def training_step(self, batch, batch_idx):
    X, y = batch
    outputs = self.mod(X)
    return self.loss_function(outputs, y)

  def configure_optimizers(self):
    return optim.Adadelta(model.parameters())

  def test_step(self, batch, batch_idx):
    X, y = batch
    outputs = self.mod(X)
    loss = self.loss_function(outputs, y)
    y, outputs = y.cpu(), outputs.cpu()
    predicted_classes = torch.max(outputs, 1)[1]
    self.log_dict({
        "test_precision": precision_score(y, predicted_classes, average='macro'),
        "test_recall": recall_score(y, predicted_classes, average='macro'),
        "test_F1": f1_score(y, predicted_classes, average='macro'),
        "test_acc": accuracy_score(y, predicted_classes),
    }, on_epoch=True)
    return loss

trainer = pl.Trainer(max_epochs=1, gpus=1)

# training
model = MnistLitModel()
trainer.fit(model=model, train_dataloaders=train_loader)

# testing
trainer.test(model=model, test_dataloaders=val_loader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params
---------------------------------------------------
0 | mod           | MnistResNet      | 11.2 M
1 | loss_function | CrossEntropyLoss | 0     
---------------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.701    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

### TODO 2: implement lightning data module

In [ ]:
class MnistDataModule(pl.LightningDataModule):
  ...
  # Code goes here


# training
model = MnistLitModel()
dm = MnistDataModule()
trainer.fit(model=model, datamodule=dm)